In [2]:
import numpy as np
import cv2 as cv
import csv
import matplotlib.pyplot as plt
import pandas as pd
import PIL
from PIL import Image
from numpy import asarray
import cv2
import os
from torchvision import transforms
from albumentations import Compose, Normalize, RandomCrop, HorizontalFlip, ShiftScaleRotate, HueSaturationValue
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models
from torchmetrics import AUROC
import io
import matplotlib.pyplot as plt
import time
import copy

from glob import glob
from tqdm import tqdm

import warnings
from sklearn.metrics import f1_score,accuracy_score,recall_score,roc_auc_score,roc_curve

In [10]:
#pip install evaluate

### Decoding RLEs into masks for training

In [3]:
def decode_rle_to_mask(rle, height, width, viz=False):
    '''
    rle : run-length as string formated (start value, count)
    height : height of the mask 
    width : width of the mask
    returns binary mask
    '''
    rle = np.array(rle.split(' ')).reshape(-1, 2)
    mask = np.zeros((height*width, 1, 3))
    if viz:
        color = np.random.rand(3)
    else:
        color = [1,1,1]
    for i in rle:
        mask[int(i[0]):int(i[0])+int(i[1]), :, :] = color

    return mask.reshape(height, width, 3)

In [4]:
df = pd.read_csv('/kaggle/input/Multiclass-segmentation/train.csv')

In [5]:
df

,ImageName,ImageWidth,ImageHeight,ClassNumber,Encoding
0,2008_006280,375,500,1,60301 3 60798 7 61296 9 61795 10 62293 12 6279...
1,2008_006280,375,500,2,94789 3 95289 6 95789 8 96288 11 96788 13 9728...
2,2008_006280,375,500,3,125167 1 125666 5 126166 6 126665 8 127165 9 1...
3,2008_006285,334,500,1,5323 4 5821 7 6319 9 6818 10 7218 5 7316 11 77...
4,2008_006285,334,500,2,51316 2 51320 1 51814 8 52313 9 52387 2 52812 ...
...,...,...,...,...,...
6779,2010_006079,375,500,3,87214 4 87711 9 88209 12 88708 14 89207 16 897...
6780,2010_006086,375,500,1,45665 5 46162 9 46603 5 46639 7 46656 16 47099...
6781,2010_006086,375,500,2,56191 3 56691 6 56718 6 57191 9 57211 28 57675...
6782,2010_006086,375,500,3,81612 2 82102 13 82589 27 83087 30 83531 22 83...


In [30]:
df = pd.read_csv('/kaggle/input/Multiclass-segmentation/train.csv')
dfcp=df.copy()
dfcp = dfcp.drop_duplicates(subset=['ImageName'], keep='first')
print('length of dfcp: ', len(dfcp))
traindata={"imgs":[],"masks":[]}
path='/kaggle/input/Multiclass-segmentation/TrainImages'
ext='.jpg'

for i in dfcp['ImageName']:
    imgpath=os.path.join(path,i+ext)
    masks=[]
    num=0
    for j in df[df['ImageName'] == i]['Encoding']:
        hi = df[df['ImageName'] == i]['ImageWidth'].values[0]
        wi = df[df['ImageName'] == i]['ImageHeight'].values[0]
        classes = int(df[df['ImageName'] == i]['ClassNumber'].values[num])
        masks.append(decode_rle_to_mask(j, int(hi), int(wi))*classes)
        num+=1
    img = cv2.imread(imgpath)  
    img_resized = cv2.resize(img, (224, 224))
    traindata['imgs'].append(np.array(img_resized, dtype=np.float16))
    mask = np.sum(masks, axis=0)[:,:,0]
    mask_resized = cv2.resize(mask, (224, 224), interpolation = cv2.INTER_NEAREST)
    traindata['masks'].append(np.array(mask_resized, dtype=np.float16))


length of dfcp:  2172


In [32]:
df_test = pd.read_csv('/kaggle/input/Multiclass-segmentation/test.csv')

In [33]:
np.unique(traindata['masks'][9])

array([0., 1., 2., 3., 4.], dtype=float16)

In [34]:
df_test

,ImageName,ImageHeight,ImageWidth,Encoding
0,2008_000008,442,500,NaN
1,2008_000009,375,500,NaN
2,2008_000019,272,480,NaN
3,2008_000026,375,500,NaN
4,2008_000053,375,500,NaN
...,...,...,...,...
977,46,1024,768,NaN
978,47,1024,768,NaN
979,48,1024,768,NaN
980,49,1024,768,NaN


In [35]:
img = cv2.imread("/kaggle/input/Multiclass-segmentation/TestImages/1.jpg")

In [36]:
print(img)

None


In [37]:
df_testcp=df_test.copy()
df_testcp = df_testcp.drop_duplicates(subset=['ImageName'], keep='first')
print('length of dfcp: ', len(df_testcp))
testdata={"imgs":[]}
path='/kaggle/input/Multiclass-segmentation/TestImages'
ext='.jpg'

value = ""
for i in df_testcp['ImageName']:
    if(len(i)) == 1:
        value = "0"
    else:
        value = ""
    imgpath=os.path.join(path,value+i+ext)
    img = cv2.imread(imgpath)
    img_resized = cv2.resize(img, (224, 224))
    testdata['imgs'].append(np.array(img_resized, dtype=np.float16))

length of dfcp:  982


In [38]:
def transform_test(img, mask):
    data_transforms = Compose([
                        HorizontalFlip(),
                       # add more
                        
                        Normalize(),
                        ToTensorV2()
                    ])
    img= img.astype("float32")
    mask = mask.astype("float32")
    sample = {
              "image":img,
              "mask": mask,
              
          }
    sample = data_transforms(**sample)
    return sample

In [39]:
# imgs_test = []
# for i in range(len(testdata['imgs'])):
#     imgs_train.append(transform(testdata['imgs'][i],testdata['masks'][i] ))


In [40]:
def transform(img, mask):
    data_transforms = Compose([
                        HorizontalFlip(),
                        # add more


                        Normalize(),
                        ToTensorV2()
                    ])
    img= img.astype("float32")
    mask = mask.astype("int64")
    sample = {
              "image":img,
              "mask": mask,
              
          }
    sample = data_transforms(**sample)
    return sample


# Notes: add more functions inside Compose for increase the segmentation. Read the documentation 

In [41]:
imgs_train = []
for i in range(len(traindata['imgs'])):
    imgs_train.append(transform(traindata['imgs'][i],traindata['masks'][i] ))


In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name())

cuda:0


In [8]:
Test_split = 100



#change the number make it 80% of data

dataloaders = {
    'train': torch.utils.data.DataLoader(imgs_train[:Test_split], batch_size=8,
                                            shuffle=True, num_workers=2),
    'test': torch.utils.data.DataLoader(imgs_train[Test_split:], batch_size=8, #sampler = sampler3,
                                            shuffle=True, num_workers=2)
                                            }

NameError: name 'imgs_train' is not defined

In [ ]:
ex_image_train = next(iter(dataloaders['train']['image']))# should be 8 * 224 * 224 * 3 because number of batches = 8
ex_mask_train = next(iter(dataloaders['train']['mask']))# should be 8 * 224 * 224 * 1 because number of batches = 8
print(ex_image_train.shape)
#Next iter just gives Example of 8 images 

In [ ]:
ex_image_test = next(iter(dataloaders['test']['image']))# should be 8 * 224 * 224 * 3 because number of batches = 8
ex_mask_test = next(iter(dataloaders['test']['mask']))# should be 8 * 224 * 224 * 1 because number of batches = 8

print(ex_image_test.shape)




next(iter()) gives you one example from itererable object it is equals = i when you loop like ig you looped on 
dataloaders: 

    For i in dataloaders:
        ex = i['train']['image']
        
        
So, next(iter()) just gives you one i, but in case your data have number of images > number of batch size(8 in our case). the iteration will iterate until number of images will be finished  

In [45]:
#!pip install transformers

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


# Custom Segformer

In [9]:
#pip install transformers

In [5]:
from transformers import SegformerModel, SegformerConfig

import torch

import torch.nn as nn

from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation

from PIL import Image

import requests

from transformers import SegformerPreTrainedModel, SegformerForSemanticSegmentation

from transformers import SegformerModel, SegformerConfig

In [6]:
class CustomSegFormer2(SegformerPreTrainedModel):
    def __init__(self, config, in_channel, n_classes=5):
        super().__init__(config)
        self.segformer = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b4-finetuned-ade-512-512", ignore_mismatched_sizes=True)
        self.logits_outputs = nn.Conv2d(in_channel, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
        self.distance_outputs = nn.Conv2d(in_channel, out_channels=1, kernel_size=1, stride=1, padding=0)
        

    def forward(self, pixel_values):
        # forward to get raw logits
        outputs = self.segformer(pixel_values) # what's the number of output channels?
        logits_outputs = self.logits_outputs(outputs[0])
        distancemap_outputs = self.distance_outputs(outputs[0])        

        # next, do whatever you want with the logits to compute loss
        # upsample logits here; the predicted distance map is also needed to unsampled, right?
        upsampled_logits = nn.functional.interpolate(logits_outputs, size=pixel_values.shape[-2:], mode="bilinear", align_corners=False)
        upsampled_distancemaps = nn.functional.interpolate(distancemap_outputs , size=pixel_values.shape[-2:], mode="bilinear", align_corners=False)
        # calculate losses for logits and distance map respectively and sum the two losses
        
        return  upsampled_logits, upsampled_distancemaps

In [7]:
from transformers import SegformerModel, SegformerConfig
import torch
import numpy as np

ex=torch.rand(8,3,224,224)

configuration = SegformerConfig()
model=CustomSegFormer2(configuration,in_channel=150)
logits, _ = model(ex)
logits.shape

torch.Size([8, 5, 224, 224])

# Output note
for the loss, put the logits and the GT directly, 
But before the argmax function for calculting IOU you have first to enter the logits into softmax function: 
ex -- > Argmax(Softmax(Logits))

# Now what is next: 
I gave you some hints, now you can start from here and open any kaggle competition to see how to run training pipline for segmentation, it is supposed also to know how to run training segmetnation pipline if you have a look on any kaggle the classification pipline. Just be comfort with dataloader variable I llustriate in the previous cells and actually they repressents


# if you have any question send to me i will be able to reply from 6:00 pm